### **Objectif**

Nous voulons déterminer la hauteur optimale de la digue $h_d$ afin de minimiser le risque d’inondation en nous basant uniquement sur les relevés historiques des hauteurs de crue.

### **Chargement des données et visualisation**

In [11]:
import pandas as pd

# Charger le fichier CSV
df =  pd.ExcelFile("Données-Projet-Pratique-Fiabilité.xlsx")
print(df.sheet_names)  
df = df.parse(df.sheet_names[0])  # Charge la première feuille 

print("\nRésumé des valeurs manquantes :")
print(df.isnull().sum())
print(df["Hauteur associée mesurée (en m)"].describe())

['Données débit-hauteur', 'Données coûts']

Résumé des valeurs manquantes :
Année                                     0
Débit mesuré maximal annuel (en m3/s)     0
Hauteur associée mesurée (en m)          26
dtype: int64
count    123.000000
mean       3.896748
std        1.272337
min        1.000000
25%        3.100000
50%        3.900000
75%        4.750000
max        7.200000
Name: Hauteur associée mesurée (en m), dtype: float64


Nous avons 26 valeurs manquantes sur la hauteur de crue.
Nous allons traiter cela en deyx façons :

- Méthode naïve qui ignore les valeurs censurées et se base uniquement sur les observations disponibles.
- Méthode des quantiles avec estimation par moyenne : Remplacement des valeurs censurées par la moyenne des hauteurs connues avant le calcul des quantiles.
- Méthode des quantiles avec estimation par régression : Modélisation de la relation entre le débit et la hauteur pour estimer les valeurs censurées avant de calculer les quantiles.

### **Méthode naïve**

Dans cette première approche, on supprime les valeurs censurées et on travaille uniquement avec les hauteurs de crue observées.
La hauteur de la digue $h_d$ est alors définie comme la valeur maximale observée augmentée d’une marge de sécurité de 5% afin de prendre en compte les incertitudes liés à cette méthode.

In [3]:
df_naive = df.dropna(subset=["Hauteur associée mesurée (en m)"])

h_max = df_naive["Hauteur associée mesurée (en m)"].max()

# Ajout d’une marge de sécurité de 10%
h_empirique = h_max * 1.05

print(f"Hauteur maximale observée : {h_max:.2f} m")
print(f"Hauteur empirique (max + 5%) : {h_empirique:.2f} m")


Hauteur maximale observée : 7.20 m
Hauteur empirique (max + 5%) : 7.56 m


**Justification :**
L’idée derrière cette approche était de fixer une hauteur de digue garantissant la protection contre toutes les crues enregistrées, tout en ajoutant une marge de sécurité de 5 % pour anticiper d’éventuelles variations futures. Cette méthode repose sur l’hypothèse que la hauteur maximale observée est un bon indicateur du risque extrême et qu’une simple majoration permettrait d’éviter toute inondation. Toutefois, cette méthode présente une limite majeure : elle ignore totalement les valeurs censurées. Cela peut donc mener à une surestimation de $h_d$ si la crue maximale observée était exceptionnelle, ou au contraire à une sous-estimation si les crues les plus extrêmes étaient absentes des données disponibles.


### **Méthode des quantiles: estimation par moyenne**
i nous utilisons directement la méthode des quantiles sans estimation, nous obtenons une sous-estimation de $h_d$ car les valeurs censurées sont ignorées.
Dans cette approche, les valeurs censurées sont estimées par la moyenne des hauteurs observées.
Une fois cette estimation réalisée, nous appliquons les quantiles 95% et 99% pour définir un seuil en fonction du risque d’inondation acceptable.

In [15]:
mean_height = df["Hauteur associée mesurée (en m)"].mean()
# Remplir les valeurs manquantes avec la moyenne
df_filled = df.copy()
df_filled["Hauteur associée mesurée (en m)"].fillna(mean_height, inplace=True)

# Calcul des quantiles 95% et 99% après avoir rempli les valeurs manquantes
h_q95 = df_filled["Hauteur associée mesurée (en m)"].quantile(0.95)
h_q99 = df_filled["Hauteur associée mesurée (en m)"].quantile(0.99)

print(f"Quantile 95% après estimation : {h_q95:.2f} m")
print(f"Quantile 99% après estimation : {h_q99:.2f} m")

Quantile 95% après estimation : 5.86 m
Quantile 99% après estimation : 6.70 m


In [16]:
hauteurs_estimees = df["Hauteur associée mesurée (en m)"].dropna().values
n = len(hauteurs_estimees)  # Taille de l'échantillon après estimation

# Approximation de l'écart-type des quantiles (Méthode Harrell-Davis)
sigma_q95 = (df["Hauteur associée mesurée (en m)"].quantile(0.975) - df["Hauteur associée mesurée (en m)"].quantile(0.025)) / 2
sigma_q99 = (df["Hauteur associée mesurée (en m)"].quantile(0.995) - df["Hauteur associée mesurée (en m)"].quantile(0.005)) / 2

# Calcul des IC à 95%
z_alpha_95 = stats.norm.ppf(0.975)
z_alpha_99 = stats.norm.ppf(0.995)
IC_95 = (h_q95 - z_alpha_95 * sigma_q95 / np.sqrt(n), h_q95 + z_alpha * sigma_q95 / np.sqrt(n))
IC_99 = (h_q99 - z_alpha_99 * sigma_q99 / np.sqrt(n), h_q99 + z_alpha * sigma_q99 / np.sqrt(n))

# Affichage des résultats
print(f"Quantile 95% après estimation (régression) : {h_q95:.2f} m")
print(f"IC à 95% pour le quantile 95% : [{IC_95[0]:.2f} m ; {IC_95[1]:.2f} m]")
print(f"Quantile 99% après estimation (régression) : {h_q99:.2f} m")
print(f"IC à 95% pour le quantile 99% : [{IC_99[0]:.2f} m ; {IC_99[1]:.2f} m]")

Quantile 95% après estimation (régression) : 5.86 m
IC à 95% pour le quantile 95% : [5.41 m ; 6.31 m]
Quantile 99% après estimation (régression) : 6.70 m
IC à 95% pour le quantile 99% : [6.02 m ; 7.22 m]


**Justification :**
Cette approche part de l’idée que, pour traiter les censures, on peut leur attribuer une valeur moyenne, ce qui permet ensuite de calculer des quantiles pour déterminer $h_d$. L’objectif était d’utiliser une méthode simple et robuste, qui s’appuie sur une hypothèse de distribution normale des hauteurs de crue. En estimant les valeurs censurées par la moyenne des observations disponibles, nous avons supposé que ces valeurs étaient réparties de manière homogène, sans être nécessairement plus élevées que les autres.

Cependant, cette approche lisse les extrêmes, ce qui a conduit à une sous-estimation des quantiles élevés. Nous avons constaté que le quantile 99 % issu de cette méthode (6.70 m) était significativement inférieur à l'approche naive, traduisant un possible biais vers le bas. Cela nous a amenés à chercher une alternative permettant une estimation plus fidèle des valeurs censurées.

### **Méthode des quantiles avec régression**
Une solution consiste à modéliser la relation entre le débit et la hauteur de crue afin d’estimer ces valeurs.
L’idée principale est que le débit Q et la hauteur H sont fortement corrélés : lorsqu’un débit important est observé, la hauteur de crue est généralement élevée. En exploitant cette relation, nous pouvons estimer les hauteurs manquantes à partir des débits mesurés.

In [5]:
import numpy as np
from sklearn.linear_model import LinearRegression

# On prend uniquement les lignes avec des valeurs observées pour entraîner le modèle
df_observed = df.dropna(subset=["Hauteur associée mesurée (en m)"])

# Modèle de régression entre le débit et la hauteur observée
X_train = df_observed["Débit mesuré maximal annuel (en m3/s)"].values.reshape(-1, 1)
y_train = df_observed["Hauteur associée mesurée (en m)"].values

# Entraînement du modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

# Prédiction des valeurs censurées
df_missing = df[df["Hauteur associée mesurée (en m)"].isna()]
X_missing = df_missing["Débit mesuré maximal annuel (en m3/s)"].values.reshape(-1, 1)
df.loc[df["Hauteur associée mesurée (en m)"].isna(), "Hauteur associée mesurée (en m)"] = model.predict(X_missing)

# Calcul des quantiles après estimation
h_q95_r = df["Hauteur associée mesurée (en m)"].quantile(0.95)
h_q99_r = df["Hauteur associée mesurée (en m)"].quantile(0.99)

print(f"Quantile 95% après estimation (régression) : {h_q95_r:.2f} m")
print(f"Quantile 99% après estimation (régression) : {h_q99_r:.2f} m")


Quantile 95% après estimation (régression) : 6.36 m
Quantile 99% après estimation (régression) : 7.37 m


In [9]:
import scipy.stats as stats

hauteurs_estimees = df["Hauteur associée mesurée (en m)"].dropna().values
n = len(hauteurs_estimees)  # Taille de l'échantillon après estimation

# Calcul des quantiles avec Pandas
h_q95_r = df["Hauteur associée mesurée (en m)"].quantile(0.95)
h_q99_r = df["Hauteur associée mesurée (en m)"].quantile(0.99)

# Approximation de l'écart-type des quantiles (Méthode Harrell-Davis)
sigma_q95 = (df["Hauteur associée mesurée (en m)"].quantile(0.975) - df["Hauteur associée mesurée (en m)"].quantile(0.025)) / 2
sigma_q99 = (df["Hauteur associée mesurée (en m)"].quantile(0.995) - df["Hauteur associée mesurée (en m)"].quantile(0.005)) / 2

# Calcul des IC à 95%
z_alpha_95 = stats.norm.ppf(0.975)
z_alpha_99 = stats.norm.ppf(0.995)
IC_95 = (h_q95_r - z_alpha_95 * sigma_q95 / np.sqrt(n), h_q95_r + z_alpha * sigma_q95 / np.sqrt(n))
IC_99 = (h_q99_r - z_alpha_99 * sigma_q99 / np.sqrt(n), h_q99_r + z_alpha * sigma_q99 / np.sqrt(n))

# Affichage des résultats
print(f"Quantile 95% après estimation (régression) : {h_q95_r:.2f} m")
print(f"IC à 95% pour le quantile 95% : [{IC_95[0]:.2f} m ; {IC_95[1]:.2f} m]")
print(f"Quantile 99% après estimation (régression) : {h_q99_r:.2f} m")
print(f"IC à 95% pour le quantile 99% : [{IC_99[0]:.2f} m ; {IC_99[1]:.2f} m]")

Quantile 95% après estimation (régression) : 6.36 m
IC à 95% pour le quantile 95% : [5.95 m ; 6.77 m]
Quantile 99% après estimation (régression) : 7.37 m
IC à 95% pour le quantile 99% : [6.66 m ; 7.92 m]


**Justification :**

Pour améliorer l’estimation des valeurs censurées, nous avons cherché à exploiter une relation physique existante entre le débit et la hauteur de crue, plutôt que de leur attribuer une valeur fixe. L’idée était d’utiliser une régression linéaire pour modéliser cette relation à partir des données observées, puis d’extrapoler les valeurs censurées.

Cette approche repose sur l’hypothèse que le débit maximal annuel est un bon prédicteur de la hauteur de crue associée, ce qui est cohérent avec la physique des crues. En estimant les hauteurs manquantes à partir de cette relation, nous avons obtenu une meilleure représentation des valeurs extrêmes, évitant l’effet de lissage de la méthode par moyenne.

### **Résultats**

| **Approche** | **Prise en compte des censures ?** | **Hauteur \( h_d \) obtenue (m)** |
|-------------|----------------------------------|------------------------------|
| **Méthode naïve (Max + 5%)** | ❌ Non | **7.56** |
| **Quantile 95% (moyenne)** | ✅ Oui (Estimation moyenne) | **5.86** |
| **Quantile 99% (moyenne)** | ✅ Oui (Estimation moyenne) | **6.70** |
| **Quantile 95% (régression)** | ✅ Oui (Estimation régression) | **6.36** |
| **Quantile 99% (régression)** | ✅ Oui (Estimation régression) | **7.37** |


Les trois approches testées aboutissent à des estimations de $h_d$ comprises entre 5.86 m et 7.56 m, mettant en évidence l'impact du traitement des valeurs censurées.

La méthode naïve (7.56 m) et l’approche par régression (7.37 m) donnent des valeurs relativement proches, ce qui suggère que la modélisation du lien entre débit et hauteur permet d’approcher les crues extrêmes de manière cohérente avec les observations historiques. En revanche, l’approche par moyenne sous-estime nettement $h_d$ (6.70 m), car elle lisse les valeurs censurées sans refléter la distribution réelle des hauteurs extrêmes.

Ainsi, le quantile 99 % estimé par régression (7.37 m) constitue un bon compromis entre sécurité et optimisation des coûts, en tenant compte des censures tout en restant plus réaliste que la méthode naïve. Si une sécurité absolue est requise, la méthode naïve reste plus conservatrice, tandis que l’approche par moyenne minimise les coûts au prix d’un risque accru. Le choix final dépend donc du niveau de protection souhaité.